## Install Tools/Libraries

In [0]:
## compilation
apt install g++ unzip zip build-essential bison flex 

## download from google drive
pip install gdown

## bazel build system
wget https://github.com/bazelbuild/bazel/releases/download/1.2.1/bazel-1.2.1-installer-linux-x86_64.sh
chmod +x bazel-1.2.1-installer-linux-x86_64.sh
## install bazel
./bazel-1.2.1-installer-linux-x86_64.sh --user

## Compile Postgres + differential-privacy library

In [0]:
## activate bazel
source /home/scriptsuser/.bazel/bin/bazel-complete.bash

In [0]:
## clone modified google differrential privacy library
git clone https://github.com/NTT123/differential-privacy.git

In [0]:
cd differential-privacy/

In [0]:
## download, compile postgresql
wget https://ftp.postgresql.org/pub/source/v11.6/postgresql-11.6.tar.bz2
tar xjf postgresql-11.6.tar.bz2
cd postgresql-11.6; ./configure; make -j2

# install postgres + add postgres user
sudo -s
make install; echo -e "abc123\nabc123\n\n\n\n\n\n\n\n" | adduser postgres; mkdir /usr/local/pgsql/data; chown postgres /usr/local/pgsql/data
exit # back to ScriptUsers

In [0]:
## go back to differential privacy directory
cd ..

In [0]:
## fix build config (from macOS -> Linux)
sed -i 's/darwin-fastbuild/k8-fastbuild/g' differential_privacy/postgres/BUILD

In [0]:
## build dp library
bazel build differential_privacy/...

In [0]:
## build and install dp extention for postgres
export PG_DIR=/usr/local/pgsql; export PATH="$PATH:$HOME/bin"; cd /content/differential-privacy; ./differential_privacy/postgres/install_extension.sh

## Run postgres server + Initialize database

In [0]:
## start postgres server under posgres user
su - postgres
/usr/local/pgsql/bin/initdb -D /usr/local/pgsql/data
/usr/local/pgsql/bin/pg_ctl -D /usr/local/pgsql/data -l logfile start
exit

In [0]:
## create a database `test`
sudo -u postgres /usr/local/pgsql/bin/createdb test

In [0]:
## download csv dataset
PATH=/home/scriptsuser/.local/bin gdown https://drive.google.com/uc?id=1_Iq_UWUngLX6nSGycqb8N5_9wBJ_cJ8E

In [0]:
## create an SQL script 

cat > init.sql <<EOL
select current_database();
create table transactions (InvoiceNo int,StockCode text,Description text,Quantity int,InvoiceDate text,UnitPrice numeric,CustomerID int,Country text);

\COPY transactions (InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country) FROM './onlineretail.csv' CSV HEADER DELIMITER ',' encoding 'windows-1251';

CREATE EXTENSION anon_func;
EOL

In [0]:
## run initial SQL script
sudo -u postgres /usr/local/pgsql/bin/psql -d test -f init.sql